# Mavlink  MavCom
> Mavlink  base class for Client and server.
> Theserver is on the companion computer and the client is on the ground station PC.

[https://mavlink.io/en/mavgen_python/](https://mavlink.io/en/mavgen_python/)
[https://www.ardusub.com/developers/pymavlink.html](https://www.ardusub.com/developers/pymavlink.html)

https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_TYPE



In [14]:
#| default_exp mavlink.test_mavcom 

In [15]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [17]:
#| export
import time, os, sys

from mavcom.logging import logging
from mavcom.utils.general import LeakyQueue

# os.environ['MAVLINK20'] == '1' should be placed in mavcom.__init__.py
assert os.environ[
           'MAVLINK20'] == '1', "Set the environment variable before from pymavlink import mavutil  library is imported"

# logging.getLogger("uav").setLevel(logging.DEBUG)
# logging.root.setLevel(logging.INFO)
import threading
import queue
import typing as typ
from pathlib import Path
from inspect import currentframe, getframeinfo
from pymavlink import mavutil
from mavcom.mavlink import Component, MAVCom, mavlink

from mavcom.logging import LogLevels
from fastcore.test import *
# from mavcom.imports import *   # TODO why is this relative import on nbdev_export?


In [18]:
# logging.getLogger("uav").setLevel(logging.INFO)

In [19]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [20]:

MAV_SYSTEM_GCS_CLIENT = 200  # GCS type client (TODO its not clear if this is correct,  255 = GCS)
MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_SYSTEM_VEHICLE = 111  # 1 = vehicle
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA
MAV_COMP_ID_CAMERA = mavutil.mavlink.MAV_COMP_ID_CAMERA
MAV_COMP_ID_USER1 = mavutil.mavlink.MAV_COMP_ID_USER1

In [21]:
show_doc(Component)

---

### Component

>      Component (source_component:int, mav_type:int, loglevel:LogLevels|int=20)

Create a client component to send commands to a companion computer or GCS that will control a cameras via a CameraServer instance

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| source_component | int |  | used for component indication |
| mav_type | int |  | used for heartbeat MAV_TYPE indication |
| loglevel | LogLevels \| int | 20 | logging level |

In [22]:
show_doc(MAVCom)

---

### MAVCom

>      MAVCom (connection_string:str, baudrate:int=57600, source_system:int=111,
>              loglevel:LogLevels|int=20)

Mavlink Base to set up a mavlink_connection for send and receive messages to and from a remote system.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| connection_string | str |  | "udpin:localhost:14550" |
| baudrate | int | 57600 | baud rate of the serial port |
| source_system | int | 111 | remote or air system   1 = vehicle |
| loglevel | LogLevels \| int | 20 | logging level |

In [23]:
# | hide
# assert False, "Stop here"

In [24]:
# show_doc(MavLinkBase)

#### Test locally using UDP ports

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [27]:

#|exports
def test_MAVCom():
    """Test MAVCom with a client and server on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`"""
    with MAVCom("udpin:localhost:14445", source_system=111, loglevel=LogLevels.DEBUG) as client:
        with MAVCom("udpout:localhost:14445", source_system=222, loglevel=LogLevels.DEBUG) as server:
            server.add_component(Component(server, mav_type=mavlink.MAV_TYPE_CAMERA, loglevel=LogLevels.DEBUG))
            client.add_component(Component(client, mav_type=mavlink.MAV_TYPE_GCS, loglevel=LogLevels.DEBUG))
    
            MAX_PINGS = 4
            client.component[11].send_ping(222, 22, max_pings=MAX_PINGS)
            time.sleep(0.5)
    
    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    
    test_eq(server.message_cnts[111]['PING'], MAX_PINGS)
    test_eq(server.message_cnts[111]['HEARTBEAT']>0, True) 
    test_eq(client.message_cnts[222]['PING'], MAX_PINGS)
    test_eq(client.message_cnts[222]['HEARTBEAT']>0, True)
    
test_MAVCom()

INFO |07.381| mavcom.MAVCom   | mavcom.py :386 |  Thread-17 | MainProces | MAVLink Mav2: True, source_system: 111
INFO |07.484| mavcom.MAVCom   | mavcom.py :386 |  Thread-18 | MainProces | MAVLink Mav2: True, source_system: 222
DEBUG|07.487| mavcom.Componen | basecompon:119 | MainThread | MainProces | set_mav_connection Component general.py:119 self.mav_com = <MAVCom>
DEBUG|07.489| mavcom.Componen | basecompon:165 |  Thread-19 | MainProces | Starting heartbeat type: 30 to all Systems and Components
DEBUG|07.490| mavcom.Componen | basecompon:168 |  Thread-19 | MainProces | Sent heartbeat 30 self.source_system = 222 self.source_component = <MAVCom>
Exception in thread Thread-19:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner
DEBUG|07.491| mavcom.Componen | basecompon:127 | MainThread | MainProces | Called from Component.start_mav_connection(), override to add startup behaviour
INFO |07.492| mavcom.Componen | basecompon:123 | Mai

KeyError: 11

In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
show_doc(MAVCom.add_component)

In [ ]:
show_doc(MAVCom.close)

This will show on wireshark as follows:
> Using the display filte string `not icmp  && udp.port eq 14445 && mavlink_proto`

![](images/wireshark_udp:14445.png)

For debugging help see [http://localhost:3000/tutorials/mavlink_doc&debug.html](http://localhost:3000/tutorials/mavlink_doc&debug.html)
 and [http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging](http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging)

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()